# Weather Company Data Disease Tracker API

This notebook shows how an application can access the Weather Company Data Disease Tracker API endpoint.

The API allows you to track the progression of a disease for a given location. It provides information regarding active diseases including confirmed cases, deaths, and recoveries over a period of up to 60 days in the past.

## Obtain a Weather Company API Key

If you're participating in the [Call for Code](https://developer.ibm.com/callforcode/) Global Challenge, go the the [special Call for Code Weather web site](https://callforcode.weather.com/) and [register](https://callforcode.weather.com/register). A time-limited API key will be sent to you via email. The documentation for the Weather Company APIs for Call for Code can be found the [here](https://callforcode.weather.com/documentation/).

## (Optional) Obtain a HERE Location Services API Key

When using the application you may pass it a geocode (i.e., '35.843686,-78.78548'), a postal key (e.g., 90210:US), or an address. If you pass an address, the application will try to use HERE Location Services for geocoding.

To access the HERE Geocoding API, an API Key is required. Follow the instructions outlined in the [HERE Developer Portal](https://developer.here.com/ref/IBM_starterkit_Covid?create=Freemium-Basic) to generate a free [API Key](https://developer.here.com/documentation/authentication/dev_guide/topics/api-key-credentials.html).


<br>

**Upgrade/install `bokeh`**

In [ ]:
!pip install --user bokeh==1.4.0 --upgrade

<br>

**Set API keys**

1. Set your Weather Company Data API key (`TWC_APIKEY`).
1. (Optional) If would like to be able to pass an address (instead of geocode or postal key), set your HERE Location Services API key (`HERE_APIKEY`).


In [ ]:
# @hidden_cell

TWC_APIKEY = '...'
HERE_APIKEY = '...'


<br>

**Add helper functions to check and/or convert location**


In [ ]:
import requests
import re

geocode_endpoint = 'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'

coordinates_regex = "^[-+]?([1-8]?\d(\.\d+)?|90(\.0+)?),\s*[-+]?(180(\.0+)?|((1[0-7]\d)|([1-9]?\d))(\.\d+)?)$"

postalkey_regex = ".+:[a-zA-Z]{2}$"


def is_postalkey (location):
    return isinstance(location, str) and re.match(postalkey_regex, location)


def is_geocode (location):
    geocode = None
    if isinstance(location, str):
        l = location.split(',')
        if len(l) == 2:
            geocode = '{},{}'.format(l[0].strip(), l[1].strip())
    elif isinstance(location, list) and len(location) == 2:
        geocode = ','.join(str(l) for l in location)
        
    if geocode is not None and re.match(coordinates_regex, geocode):
        return [float(l) for l in location.split(',')]
    else:
        return False
    

def get_geocode (address):
    g = is_geocode(address)

    if not g:
        url = geocode_endpoint.format(address=address, api_key=HERE_APIKEY)
        response = requests.get(url)

        if response.ok:
            jsonResponse = response.json()
            position = jsonResponse['items'][0]['position']
            g = [position['lat'], position['lng']]
        else:
            print(response.text)
            
    return g


<br>

**Add function to query TWC API to get most recent report**


In [ ]:
import requests

# location_type = county, state, or country
# duration = '60day'
disease_endpoint = 'https://api.weather.com/v3/wx/disease/tracker/{location_type}/{duration}?format=json&apiKey={api_key}&{location_param}={location}'


def get_covid_stats (location, location_type='county', duration='60day'):
    loc = location if is_postalkey(location) else ','.join(map(str, location))

    url = disease_endpoint.format(
        location_type=location_type,
        duration=duration,
        api_key=TWC_APIKEY,
        location_param='postalKey' if is_postalkey(location) else 'geocode',
        location=loc
    )

    response = requests.get(url)

    if response.ok:
        jsonResponse = response.json()
        covid_stats = jsonResponse['covid19']
        return covid_stats
    else:
        print(response.text)
        return {}
    

<br>

**Add function to plot the report received from TWC API**


In [ ]:
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.io import output_notebook

import numpy as np
import pandas as pd

output_notebook()

def get_stats_plot(stats):
    last_zero_day = stats['confirmed'].index(0) if 0 in stats['confirmed'] else len(stats['confirmed'])
    
    df = pd.DataFrame(stats, columns=['dateReport', 'confirmed', 'deaths'])[:last_zero_day + 1]
    df['date'] = df['dateReport'].astype(np.datetime64)
    
    source = ColumnDataSource(df)
    
    plot = figure(plot_width=800, plot_height=600, x_axis_type='datetime')

    confirmed_line = plot.line(source=source, x='date', y='confirmed', line_color='orange', line_width=2, legend_label='confirmed')
    deaths_line = plot.line(source=source, x='date', y='deaths', line_color='red', line_width=2, legend_label='deaths')
    
    plot.title.text = 'Total COVID-19 reported cases by date for {} (Pop: {})'.format(stats['recordLocation'], stats['totalPopulation'])
    plot.legend.location = 'top_left'
    plot.xaxis.axis_label = 'date'
    plot.yaxis.axis_label = 'reported cases'
    plot.xaxis.ticker.desired_num_ticks = len(df) // 2
    
    hover = HoverTool(
        tooltips=[
            ('date', '@dateReport'),
            ('confirmed', '@confirmed'),
            ('deaths', '@deaths')
        ],
        mode='vline'
    )
    hover.renderers = [confirmed_line]

    plot.add_tools(hover)
    
    return plot


In [ ]:
from bokeh.io import show

def show_progression_plot(address, report_type='county'):
    geocode = get_geocode(address)
    stats = get_covid_stats(geocode, location_type=report_type)
    print(stats)
    plot = get_stats_plot(stats)
    show(plot)

def show_progress_table(address, report_type='county'):
    geocode = get_geocode(address)
    stats = get_covid_stats(geocode, location_type=report_type)
    df = pd.DataFrame(stats, columns=['dateReport', 'confirmed', 'deaths'])
    print(df)


<br>

**Run**


In [ ]:
show_progress_table('02109:US')
# show_progress_table('42.3584, -71.0598')
# show_progress_table('boston, ma')

In [ ]:
show_progression_plot('New York, NY')

## Optional

The following section required the `ipywidgets` installed

https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [ ]:
# from ipywidgets import interactive

# def show_interactive_progression_plot():
#     return interactive(
#         show_progression_plot,
#         {'manual': True},
#         address=('Raleigh, NC'),
#         report_type=['county', 'state']
#     )


In [ ]:
# show_interactive_progression_plot()